In [ ]:
#lancer la communication asynchrone avec thymio
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
# Premade libraries
import cv2
import time
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Polygon, Point, LineString

In [ ]:
# Homemade functions
import Vision.vision as vs
import Global_Navigation.robot as rbt
import Global_Navigation.global_navigation as glb
import Motion_Control.thymio_control as ctrl

In [ ]:
cap = cv2.VideoCapture(1)
vs.get_image(cap)
cap.release()

In [ ]:
# Main
# Data
margin = 30

# Initialisation
# Create an instance of our robot
thymio = rbt.RobotNav()
# Start the video capture
cap = cv2.VideoCapture(0)

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if thymio.get_state() == 0:
        thymio.initialisation_step(frame, margin, True)
        thymio.increase_step()
        thymio.set_state(1)
    elif thymio.get_state() == 1:
        # We use the position determined by the camera
        position, st = vs.detect_start(frame, False, False)
        frame = glb.draw_thymio(frame, position)
        frame = glb.draw_path(frame, thymio.get_path())
        # We update the position of the robot
        if position is not None:
            thymio.update_position_cam(st)
        else:
            thymio.update_position_kalman()
        """
        # We check if the robot encounter an obstacle
            to be implemented
        """
        # We try to reach the next goal
        geometry = thymio.get_geometry()
        step = thymio.get_crt_step()
        path = thymio.get_path()
        #ctrl.astolfi((geometry[0], geometry[1]), geometry[2], path[step], node) 
        #thymio.update_step_respo(15, False)

    elif thymio.get_state() == 2:
        ctrl.stop_motors(node)
        ctrl.leds_blink(node)
        print('out')
        break # Exits the code


    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
ctrl.set_motor_speed(100, -100, node)

In [ ]:
ctrl.stop_motors(node)